# report

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import copy
from contract_select import contract_select
from hedge_calc import hedge_calc
from greek_calc import greek_calc, pnl_daily_calc
from position_adj_cost import position_adj_cost

In [ ]:
main_original = contract_select(select_type='main', date= '2015-12-10', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.08, minDTE=15, num=3)
hedge_original = contract_select(select_type='hedge', date= '2015-12-10', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.06, minDTE=15, num=3)
main_original.update(hedge_original)

position = hedge_calc(main_original)

position_initial = copy.deepcopy(position)
position_initial['main']['WEIGHT'] = position_initial['main']['WEIGHT'] * 2
position_initial['hedge']['WEIGHT'] = position_initial['hedge']['WEIGHT'] * 2
position_initial['emini']['WEIGHT'] = position_initial['emini']['WEIGHT'] * 2
position_adj_cost(position, position_initial)

In [ ]:
greek_pnl_daily = pd.DataFrame()
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = 1

d = copy.deepcopy(position)
daily = []
for i in range(20):
    try:
        greek_pnl_daily = greek_pnl_daily.append(greek_calc(d))
        d = copy.deepcopy(pnl_daily_calc(d))
        greek_pnl_daily['daily_pnl'][-1] =  d['daily_rtn']
        daily.append(d)
        print(d['main'].index[0][0], d['main']['[DTE]'][0], d['daily_rtn'])
        temp = copy.deepcopy(hedge_calc(d))
        greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(d, temp)['adjust_cost']
        d=copy.deepcopy(temp)
    except: break

In [ ]:
greek_pnl_daily